# ***STAGE 2 - DATA PRE-PROCESSING***

## ***Kelompok 7 - Lucky 8***

### Nama Anggota Kelompok :
1. Arif Budi Utomo;
2. M. Rivaldi Anwar Putra;
3. Natasya Indah;
4. Fathimah Azzahra Albaar;
5. Zahky Awal;
6. Dony Bagaskara;
7. Rahmawati;
8. Venessa.

# ***SOAL***

## ***1. Data Cleansing***

Lakukan pembersihan data, sesuai yang diajarkan di kelas, seperti\: <br>
A. Handle missing values <br>
B. Handle duplicated data <br>
C. Handle outliers <br>
D. Feature transformation <br>
E. Feature encoding <br>
F. Handle class imbalance <br>

Di laporan homework, tuliskan apa saja yang telah dilakukan dan metode yang digunakan.

* Tetap tuliskan jika memang ada tidak yang perlu di-handle (contoh: “Tidak perlu feature encoding karena semua feature sudah numerical” atau “Outlier tidak di-handle karena akan fokus menggunakan model yang robust terhadap outlier”).

## ***2. Feature Engineering***

Cek feature yang ada sekarang, lalu lakukan: <br>
A. Feature selection (membuang feature yang kurang relevan atau redundan) <br>
B. Feature extraction (membuat feature baru dari feature yang sudah ada) <br>
C. Tuliskan minimal 4 feature tambahan (selain yang sudah tersedia di dataset) yang mungkin akan sangat membantu membuat performansi model semakin bagus (ini hanya ide saja, untuk menguji kreativitas teman-teman, tidak perlu benar-benar dicari datanya dan tidak perlu diimplementasikan) <br>

* Untuk 2A & 2B, tetap tuliskan jika memang tidak bisa dilakukan (contoh: “Semua feature digunakan untuk modelling (tidak ada yang dihapus), karena semua feature relevan”)

# ***JAWABAN***

## ***Import Libraries***

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn import under_sampling, over_sampling

print('Numpy Version : ', np.__version__)
print('Pandas Version : ', pd.__version__)
print('Seaborn Version : ', sns.__version__)

Numpy Version :  1.23.4
Pandas Version :  1.5.2
Seaborn Version :  0.12.1


### ***Mengubah Parameter Default Matpotlib***

In [2]:
from matplotlib import rcParams

rcParams['figure.figsize'] = 12, 4
rcParams['lines.linewidth'] = 3
rcParams['xtick.labelsize'] = 'x-large'
rcParams['ytick.labelsize'] = 'x-large'

## ***Load Dataset***

In [3]:
df = pd.read_csv('mc_eda.csv')

In [4]:
df.sample(5)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Age,Total_amount_spent,Total_accept_campaign
1553,879,1976,Master,Married,49681.0,0,2,2013-11-04,66,411,...,0,0,0,0,3,11,0,44,47,0
346,2579,1957,Graduation,Married,71113.0,0,1,2013-12-17,95,495,...,0,0,0,0,3,11,0,63,340,0
511,203,1975,Master,Single,81169.0,0,0,2013-04-14,47,1288,...,0,0,0,0,3,11,0,45,801,0
573,1160,1970,Graduation,Married,13260.0,1,1,2013-08-23,48,9,...,0,0,0,0,3,11,0,50,40,0
250,2227,1958,Master,Together,26490.0,0,0,2012-09-22,92,45,...,0,0,0,0,3,11,0,62,165,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     2229 non-null   int64  
 1   Year_Birth             2229 non-null   int64  
 2   Education              2229 non-null   object 
 3   Marital_Status         2229 non-null   object 
 4   Income                 2229 non-null   float64
 5   Kidhome                2229 non-null   int64  
 6   Teenhome               2229 non-null   int64  
 7   Dt_Customer            2229 non-null   object 
 8   Recency                2229 non-null   int64  
 9   MntWines               2229 non-null   int64  
 10  MntFruits              2229 non-null   int64  
 11  MntMeatProducts        2229 non-null   int64  
 12  MntFishProducts        2229 non-null   int64  
 13  MntSweetProducts       2229 non-null   int64  
 14  MntGoldProds           2229 non-null   int64  
 15  NumD

Pengamatan\:
1. Data terdiri dari `2229` baris;
2. Sudah tidak ada nilai null di setiap kolom (`Non-Null Count < jumlah baris`), nilai null sudah dihandle saat EDA kemarin; dan
3. Sepertinya terdapat issue yang mencolok pada tipe data untuk kolom `Dt_Customer`, seharusnya tipe datanya bukan object namun `datetime`.
4. Kolom bertipe kategori : `'Education'` dan `'Marital_Status'`, nanti akan `diencoding`.
5. Kolom `'Kidhome'` dan `'Teenhome'` akan kami jadikan satu untuk meringkas dimensi.
6. Kolom `'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds'` (atau kedepannya akan kita sebut `himpunan kolom 'amount_spent'`) dan kolom `'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5'` (atau kedepannya akan kita sebut `himpunan kolom 'cmp'`) telah kami gabungkan menjadi `'Total_accept_campaign'` saat EDA kemarin.
7. Kolom `'Year_birth'` juga telah kita rubah menjadi kolom `'Age'` saat EDA kemarin.

## ***Feature Extraction (2.B)***

In [6]:
# Mengcopy Dataset
dfa = df.copy()

In [7]:
# Merubah tipe data Dt_Customer
dfa['Dt_Customer']= pd.to_datetime(dfa['Dt_Customer'])


dfa['Year'] = dfa["Dt_Customer"].dt.year
dfa['Month'] = dfa["Dt_Customer"].dt.month

In [8]:
# Menggabungkan kolom Kidhome dan Teenhome menjadi children
dfa['children'] = dfa['Kidhome'] + dfa['Teenhome']

## ***Feature Encoding (1.E)***

***Pengelompokkan Kolom***

In [9]:
# pengelompokan kolom berdasarkan jenisnya

cats = ['Education', 'Marital_Status']
nums = ['Year_Birth', 'Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response','Age','Total_amount_spent', 'Total_accept_campaign']
timestamp = ['Dt_Customer']


# pengelompokkan lebih spesifik untuk 'nums'

amount_spent = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
num_purchases = ['NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases']
cmp = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response']
others1 = ['Year_Birth', 'Income', 'Kidhome', 'Teenhome', 'Recency']
others2 = ['Complain', 'NumWebVisitsMonth', 'Age', 'Total_amount_spent', 'Total_accept_campaign']
dt_cust = ['Year', 'Month']

In [10]:
# Melihat value unique kolom Education dan Marital_Status
for col in cats:
  print(f'value counts of column {col}')
  print(dfa[col].value_counts())
  print('---'*10, '\n')

value counts of column Education
Graduation    1124
Master         570
PhD            481
Basic           54
Name: Education, dtype: int64
------------------------------ 

value counts of column Marital_Status
Married     861
Together    577
Single      484
Divorced    307
Name: Marital_Status, dtype: int64
------------------------------ 



Pengamatan : <br>
1. Kolom `'Education'` memiliki nilai bertingkat (tipe ordinal), maka untuk proses encodingnya menggunakan `Label Encoding`.
2. Kolom `'Marital_Status'` memiliki nilai yang tidak bertingkat (tipe nominal), maka untuk proses encodingnya menggunakan `One Hot Encoder`.

In [ ]:
# Distribusi 'MntWines' Asli
sns.kdeplot(dfa['MntWines'])

In [ ]:
# Distribusi 'MntWines' setelah Log Transformation
sns.kdeplot(np.log(dfa['MntWines']))

In [ ]:
# Distribusi 'MntFruits' Asli
sns.kdeplot(dfa['MntFruits'])

In [ ]:
# Distribusi 'MntFruits' setelah Log Transformation
sns.kdeplot(np.log(dfa['MntFruits']))

In [ ]:
# Distribusi 'MntMeatProducts' Asli
sns.kdeplot(dfa['MntMeatProducts'])

In [ ]:
# Distribusi 'MntMeatProducts' setelah Log Transformation
sns.kdeplot(np.log(dfa['MntMeatProducts']))

In [ ]:
# Distribusi 'MntFishProducts' Asli
sns.kdeplot(dfa['MntFishProducts'])

In [ ]:
# Distribusi 'MntFishProducts' setelah Log Transformation
sns.kdeplot(np.log(dfa['MntFishProducts']))

In [ ]:
# Distribusi 'MntSweetProducts' Asli
sns.kdeplot(dfa['MntSweetProducts'])

In [ ]:
# Distribusi 'MntSweetProducts' setelah Log Transformation
sns.kdeplot(np.log(dfa['MntSweetProducts']))

In [ ]:
# Distribusi 'MntGoldProds' Asli
sns.kdeplot(dfa['MntGoldProds'])

In [ ]:
# Distribusi 'MntGoldProds' setelah Log Transformation
sns.kdeplot(np.log(dfa['MntGoldProds']))

In [ ]:
# Distribusi 'Total_amount_spent' Asli
sns.kdeplot(dfa['Total_amount_spent'])

In [ ]:
# Distribusi 'Total_amount_spent' setelah Log Transformation
sns.kdeplot(np.log(dfa['Total_amount_spent']))

In [ ]:
# kita tranformasi
dfa['log_wines'] = np.log(dfa['MntWines'])
dfa['log_fruits'] = np.log(dfa['MntFruits'])
dfa['log_meats'] = np.log(dfa['MntMeatProducts'])
dfa['log_fish'] = np.log(dfa['MntFishProducts'])
dfa['log_sweet'] = np.log(dfa['MntSweetProducts'])
dfa['log_gold'] = np.log(dfa['MntGoldProds'])
dfa['log_spent'] = np.log(dfa['Total_amount_spent'])

In [11]:
# Mapping Education
mapping_education = {
    'Basic' : 0,
    'Graduation' : 1,
    'Master' : 2,
    'PhD' : 3
}

dfa['Education'] = dfa['Education'].map(mapping_education)

In [12]:
# One hot encoder Marital_Status
# WARNING: run code ini sekali saja
onehots = pd.get_dummies(dfa['Marital_Status'], prefix='Status')
dfa = dfa.join(onehots)

In [13]:
# Melihat keberhasilan feature encoding dan extraction
dfa.sample(5)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,Age,Total_amount_spent,Total_accept_campaign,Year,Month,children,Status_Divorced,Status_Married,Status_Single,Status_Together
476,5300,1973,2,Married,38961.0,1,0,2013-12-30,60,19,...,47,51,0,2013,12,1,0,1,0,0
1192,7342,1961,2,Divorced,59184.0,0,1,2012-09-09,6,341,...,59,731,0,2012,9,1,1,0,0,0
182,10796,1984,2,Married,30477.0,1,0,2014-01-22,16,47,...,36,19,0,2014,1,1,0,1,0,0
1077,6072,1970,2,Single,75345.0,0,0,2013-02-02,16,918,...,50,1169,1,2013,2,0,0,0,1,0
261,2387,1952,2,Together,43776.0,1,1,2013-02-03,9,177,...,68,98,0,2013,2,2,0,0,0,1


## ***Feature Selection (2.A)***

Untuk membuat model prediction response customer terhadap campaign yang akan datang, maka kami memutuskan untuk melakukan hal berikut :
1. kolom `'Year_Birth'` akan kami hapus, dan kami akan memakai kolom `'Age'` saja.
2. Kolom `'ID', 'Z_CostContact'`, dan `'Z_Revenue'` akan kami drop karena tidak berguna dalam memprediksi response.
3. Kolom `'Total_accept_campaign'` akan kami pakai, dan himpunan kolom `'cmp'` akan kita drop.
4. Kolom `'Kidhome'` dan `'Teenhome'` kita akan hapus karena sudah disatukan menjadi kolom `'child'`
5. Kolom `'Dt-Customer'` kami juga akan drop, hanya  memakai kolom `'Year'` saja, sedangkan kolom `'Month'` akan kami hapus juga.
6. Kolom `'Total_amount_spent'` kami akan pakai, sedangkan himpunan `'amount_spent`' nya akan kami drop.
7. Kolom `'Marital_Status'` juga kami drop karena sudah di encoding.

In [14]:
# Menghapus kolom yang tidak akan digunakan
dfa = dfa.drop(columns = ['ID', 'Year_Birth', 'Marital_Status', 'Kidhome', 'Teenhome', 'Dt_Customer', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Z_CostContact', 'Z_Revenue', 'Month'])

In [15]:
# Melihat keberhasilan feature Selection
dfa.sample(5)

,Education,Income,Recency,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response,Age,Total_amount_spent,Total_accept_campaign,Year,children,Status_Divorced,Status_Married,Status_Single,Status_Together
586,3,39684.0,41,2,2,1,2,7,1,1,36,62,0,2012,1,0,0,1,0
1146,1,83145.0,14,1,5,9,11,2,0,1,46,1056,1,2012,0,0,0,0,1
815,1,57091.0,0,1,7,3,7,5,0,1,59,113,1,2014,0,0,0,1,0
1808,1,72282.0,70,1,5,7,9,7,0,1,69,828,0,2013,0,0,0,1,0
1467,1,39898.0,20,2,3,0,4,7,0,0,66,65,0,2013,1,0,0,0,1


## ***Split Data Train dan Test***

Untuk memulai sesi Pre-processing, kami akan memisahkan data Train dan Test terlebih dahulu. Dan yang akan kami pre-processing adalah data train saja.

In [16]:
# Melakukan Split Dataset
train, test = train_test_split(dfa, test_size=0.3, random_state=42)

In [20]:
# Melihat jumlah data train
train.shape

(1560, 19)

In [21]:
# Melihat jumlah data train
test.shape

(669, 19)

In [22]:
train.to_csv('mc_train.csv')
test.to_csv('mc_test.csv')

In [ ]:
# Split Data Train dan Test
from sklearn.model_selection import train_test_split
train, test = train_test_split(dfa,test_size=1/3, random_state=42)

In [ ]:
# Melihat Sample Data Train
train.sample(5)

In [ ]:
# Melihat Sample Data Test
test.sample(5)

In [ ]:
# Mencari Nilai Null
train.isna().sum()

Dataset sudah bersih, tidak ada missing value (pembersihan missing value sudah dilakukan saat EDA)

In [ ]:
# cek jumlah duplicated rows
# dari semua kolom
train.duplicated().sum()

Dataset sudah tidak ada duplikat.

In [ ]:
train.info()

In [ ]:
# pengelompokan ulang kolom berdasarkan jenisnya

cats = ['Education']
nums = ['Year_Birth', 'Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'log_wines', 'log_fruits', 'log_meats', 'log_fish', 'log_sweet', 'log_gold', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth','Age','log_spent', 'Total_amount_spent', 'Total_accept_campaign', 'deal_norm', 'web_norm', 'catalog_norm', 'store_norm', 'web_visit_norm', 'log_spent_norm', 'income_std', 'recency_std', 'age_std', 'Year', 'Month']
boolean = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Complain', 'Marital_Status_Divorced', 'Marital_Status_Married', 'Marital_Status_Single', 'Marital_Status_Together', 'Response']

# pengelompokkan lebih spesifik untuk 'nums'

amount_spent = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'log_wines', 'log_fruits', 'log_meats', 'log_fish', 'log_sweet', 'log_gold']
num_purchases = ['NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases']
norm = ['deal_norm', 'web_norm', 'catalog_norm', 'store_norm', 'web_visit_norm', 'log_spent_norm']
std = ['income_std', 'recency_std', 'age_std']
others1 = ['Year_Birth', 'Income', 'Kidhome', 'Teenhome', 'Recency', 'NumWebVisitsMonth']
others2 = ['Age', 'log_spent', 'Total_amount_spent', 'Total_accept_campaign', 'Year', 'Month']

### 1) Melihat Outlier

In [ ]:
for i in range (0, len(amount_spent)) :
    plt.subplot(4,3, i+1)
    sns.boxplot(x=train[amount_spent[i]], color='green')
    plt.tight_layout()

In [ ]:
for i in range (0, len(num_purchases)) :
    plt.subplot(2,3, i+1)
    sns.boxplot(x=train[num_purchases[i]], color='green')
    plt.tight_layout()

In [ ]:
for i in range (0, len(norm)) :
    plt.subplot(2,3, i+1)
    sns.boxplot(x=train[norm[i]], color='green')
    plt.tight_layout()

In [ ]:
for i in range (0, len(std)) :
    plt.subplot(2,3, i+1)
    sns.boxplot(x=train[std[i]], color='green')
    plt.tight_layout()

In [ ]:
for i in range (0, len(others1)) :
    plt.subplot(2,3, i+1)
    sns.boxplot(x=train[others1[i]], color='green')
    plt.tight_layout()

In [ ]:
for i in range (0, len(others2)) :
    plt.subplot(2,3, i+1)
    sns.boxplot(x=train[others2[i]], color='green')
    plt.tight_layout()

### 2) Menghapus Outlier

In [ ]:
outlier = ['deal_norm', 'web_norm', 'catalog_norm', 'web_visit_norm', 'log_spent_norm', 'log_gold']
from scipy import stats

print(f'Jumlah baris sebelum memfilter outlier: {len(train)}')

filtered_entries = np.array([True] * len(train))

for col in outlier:
    zscore = abs(stats.zscore(train[col])) # hitung absolute z-scorenya
    filtered_entries = (zscore < 3) # keep yang kurang dari 3 absolute z-scorenya
    
train = train[filtered_entries] # filter, cuma ambil yang z-scorenya dibawah 3

print(f'Jumlah baris setelah memfilter outlier: {len(train)}')

In [ ]:
# pemisahan features vs target pada data training
x_train = train.drop(columns=['ID','Z_CostContact', 'Z_Revenue', 'Response'])
y_train = train[['Response']]
print(x_train.shape)
print(y_train.shape)

In [ ]:
# pemisahan features vs target pada data testing
x_test = test.drop(columns=['ID','Z_CostContact', 'Z_Revenue', 'Response'])
y_test = test[['Response']]
print(x_test.shape)
print(y_test.shape)

In [ ]:
# Melihat keadaan Target
y_train['Response'].value_counts()

In [ ]:
from imblearn import under_sampling, over_sampling
x_train_under, y_train_under = under_sampling.RandomUnderSampler(sampling_strategy = 'auto').fit_resample(x_train, y_train)
x_train_over, y_train_over = over_sampling.RandomOverSampler(sampling_strategy = 'auto').fit_resample(x_train, y_train)
x_train_over_SMOTE, y_train_over_SMOTE = over_sampling.SMOTE(sampling_strategy = 'auto').fit_resample(x_train, y_train)

In [ ]:
print('Original')
print(y_train.value_counts())
print('\n')
print('UNDERSAMPLING')
print(y_train_under.value_counts())
print('\n')
print('OVERSAMPLING')
print(y_train_over.value_counts())
print('\n')
print('SMOTE')
print(y_train_over_SMOTE.value_counts())

Kami akan menggunakan data train yang sudah di-SMOTE, karena dalam metode SMOTE dataset ditambah dengan data sintesis.

In [ ]:
# Mengecek kembali korelasi
fig, ax = plt.subplots(figsize=(20,20))
sns.heatmap(round(x_train_over_SMOTE.corr(), 2), annot = True)